# 0. Przygotowanie notebooka

In [ ]:
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 8)

# 1. Porównanie embeddingów

## 1.1 Importy i definicje

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

In [ ]:
def plot_colorless(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.scatter(X[i, 0], X[i, 1])

    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

In [ ]:
def plot_colorful(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.scatter(X[i, 0], X[i, 1], color=plt.cm.Set1(y[i] / 10.))

    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

In [ ]:
def plot_with_labels(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(digits.target[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    shown_images = np.array([[1., 1.]])
    for i in range(digits.data.shape[0]):
        dist = np.sum((X[i] - shown_images) ** 2, 1)
        if np.min(dist) < 4e-3:
            # don't show points that are too close
            continue
        shown_images = np.r_[shown_images, [X[i]]]
        imagebox = offsetbox.AnnotationBbox(
            offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r), X[i])
        ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])

In [ ]:
def plot_embedding(X):
    plot_colorless(X)
    plot_colorful(X)
    plot_with_labels(X)

## 1.2 Załadowanie danych

In [ ]:
digits = datasets.load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape
n_neighbors = 30

In [ ]:
print(digits['DESCR'])

In [ ]:
n_img_per_row = 20
img = np.zeros((10 * n_img_per_row, 10 * n_img_per_row))
for i in range(n_img_per_row):
    ix = 10 * i + 1
    for j in range(n_img_per_row):
        iy = 10 * j + 1
        img[ix:ix + 8, iy:iy + 8] = X[i * n_img_per_row + j].reshape((8, 8))

plt.imshow(img, cmap=plt.cm.binary)
plt.xticks([])
plt.yticks([])
plt.title('A selection from the 64-dimensional digits dataset')

## 1.3 Wizualizacja różnych embeddingów

Random Projection

In [ ]:
rp = random_projection.SparseRandomProjection(n_components=2)
X_projected = rp.fit_transform(X)
plot_embedding(X_projected)

Principal Components projection

In [ ]:
X_pca = decomposition.TruncatedSVD(n_components=2).fit_transform(X)
plot_embedding(X_pca)

Linear Discriminant projection

In [ ]:
X2 = X.copy()
X2.flat[::X.shape[1] + 1] += 0.01  # Make X invertible
X_lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components=2).fit_transform(X2, y)
plot_embedding(X_lda)

Isomap projection

In [ ]:
X_iso = manifold.Isomap(n_neighbors, n_components=2).fit_transform(X)
plot_embedding(X_iso)

Locally Linear Embedding

In [ ]:
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2, method='standard')
X_lle = clf.fit_transform(X)
plot_embedding(X_lle)

Modified Locally Linear Embedding

In [ ]:
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2, method='modified')
X_mlle = clf.fit_transform(X)
plot_embedding(X_mlle)

Hessian Locally Linear Embedding

In [ ]:
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2, method='hessian')
X_hlle = clf.fit_transform(X)
plot_embedding(X_hlle)

Local Tangent Space Alignment

In [ ]:
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2, method='ltsa')
X_ltsa = clf.fit_transform(X)
plot_embedding(X_ltsa)

MDS embedding

In [ ]:
clf = manifold.MDS(n_components=2, n_init=1, max_iter=100)
X_mds = clf.fit_transform(X)
plot_embedding(X_mds)

Random forest embedding

In [ ]:
hasher = ensemble.RandomTreesEmbedding(n_estimators=200, max_depth=5)
X_transformed = hasher.fit_transform(X)
pca = decomposition.TruncatedSVD(n_components=2)
X_reduced = pca.fit_transform(X_transformed)
plot_embedding(X_reduced)

Spectral embedding

In [ ]:
embedder = manifold.SpectralEmbedding(n_components=2, eigen_solver="arpack")
X_se = embedder.fit_transform(X)
plot_embedding(X_se)

# Task 1
## Użyj sklearn.manifold z biblioteki scikit-learn, aby wygenerować embedding TSNE

In [ ]:
tsne = # TODO
X_tsne = # TODO
plot_embedding(X_tsne)

# 2. Wizualizacja działania t-SNE

## 2.1 Importy

In [ ]:
from numpy import linalg
from numpy.linalg import norm
from scipy.spatial.distance import squareform, pdist

import sklearn
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.utils.extmath import _ravel
from sklearn.manifold.t_sne import (_joint_probabilities, _kl_divergence)

import matplotlib.patheffects as PathEffects
import matplotlib

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})

from moviepy.video.io.bindings import mplfig_to_npimage
import moviepy.editor as mpy

![title](images/imports.jpg)

## 2.2 Załadowanie danych

In [ ]:
digits = load_digits()
digits.data.shape

## 2.3 Test wizualizacji

In [ ]:
def scatter(x, colors):
    palette = np.array(sns.color_palette("hls", 10))
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    txts = []
    for i in range(10):
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [ ]:
X = np.vstack([digits.data[digits.target==i] for i in range(10)])
y = np.hstack([digits.target[digits.target==i] for i in range(10)])
digits_proj = TSNE().fit_transform(X) 
scatter(digits_proj, y)
plt.show()

# Task 2
## Zaimplemetuj metodę wykorzystując wzór 
## $p_{i|j} = \frac{\exp{(-||x_i-x_j||^2 / 2\sigma_i^2)}}{\sum_{k\neq j}\exp{(-||x_i-x_k||^2 / 2\sigma_i^2)}}$

In [ ]:
def _joint_probabilities_constant_sigma(D, sigma): 
    # TODO

![title](images/math.jpg)

## 2.4 Wykres odległości

In [ ]:
D = pairwise_distances(X, squared=True)
P_constant = _joint_probabilities_constant_sigma(D, 500)
P_binary = _joint_probabilities(D, 30., False)
P_binary_s = squareform(P_binary)

In [ ]:
plt.figure(figsize=(12, 36))
pal = sns.light_palette("blue", as_cmap=True)

plt.subplot(311)
plt.imshow(D[::10, ::10], interpolation='none', cmap=pal)
plt.axis('off')
plt.title("Distance matrix", fontdict={'fontsize': 16})

plt.subplot(312)
plt.imshow(P_constant[::10, ::10], interpolation='none', cmap=pal)
plt.axis('off')
plt.title("$p_{j|i}$ (constant $\sigma$)", fontdict={'fontsize': 16})

plt.subplot(313)
plt.imshow(P_binary_s[::10, ::10], interpolation='none', cmap=pal)
plt.axis('off')
plt.title("$p_{j|i}$ (variable $\sigma$)", fontdict={'fontsize': 16})
plt.show()

## 2.4 Przechwycenie pozycji z gradient descent

In [ ]:
# sklearn.manifold.t_sne._gradient_descent?

# Task 3
## Zapisz w liście positions wszystkie pośrednie pozycje punktów
Podpowiedź: użyj metody .copy()

In [ ]:
positions = []

def _gradient_descent(objective, p0, it, n_iter, n_iter_without_progress=30,
                      momentum=0.5, learning_rate=1000.0, min_gain=0.01,
                      min_grad_norm=1e-7, min_error_diff=1e-7, verbose=0,
                      objective_error=None, n_iter_check=1,
                      args=None, kwargs=None):
    
    min_grad_norm = 1e-7
    p = p0.copy().ravel()
    update = np.zeros_like(p)
    gains = np.ones_like(p)
    error = np.finfo(np.float).max
    best_error = np.finfo(np.float).max
    best_iter = 0

    for i in range(it, n_iter):
        # TODO
        
        new_error, grad = objective(p, *args)
        error_diff = np.abs(new_error - error)
        error = new_error
        grad_norm = linalg.norm(grad)

        if error < best_error:
            best_error = error
            best_iter = i
        elif i - best_iter > n_iter_without_progress:
            break
            
        if min_grad_norm >= grad_norm:
            break
            
        if min_error_diff >= error_diff:
            break

        inc = update * grad >= 0.0
        dec = np.invert(inc)
        gains[inc] += 0.05
        gains[dec] *= 0.95
        np.clip(gains, min_gain, np.inf)
        grad *= gains
        update = momentum * update - learning_rate * grad
        p += update

    return p, error, i

sklearn.manifold.t_sne._gradient_descent = _gradient_descent

## 2.5 Animacja t-SNE

In [ ]:
X_proj = TSNE().fit_transform(X)
X_iter = np.dstack(position.reshape(-1, 2) for position in positions)

In [ ]:
speed = 10.
f, ax, sc, txts = scatter(X_iter[..., -1], y)

def make_frame_mpl(t):
    i = int(t*speed)
    x = X_iter[..., i]
    sc.set_offsets(x)
    for j, txt in zip(range(10), txts):
        xtext, ytext = np.median(x[y == j, :], axis=0)
        txt.set_x(xtext)
        txt.set_y(ytext)
    return mplfig_to_npimage(f)

animation = mpy.VideoClip(make_frame_mpl, duration=X_iter.shape[2]/speed)
animation.write_videofile("tsne.webm", fps=20)

In [ ]:
mpy.ipython_display("tsne.webm", fps=20, autoplay=True)

## 2.6 Animacja podobieństwa

In [ ]:
speed = 80.

n = 1. / (pdist(X_iter[..., -1], "sqeuclidean") + 1)
Q = n / (2.0 * np.sum(n))
Q = squareform(Q)

f = plt.figure(figsize=(6, 6))
ax = plt.subplot(aspect='equal')
im = ax.imshow(Q, interpolation='none', cmap=pal)
plt.axis('tight')
plt.axis('off')

def make_frame_mpl(t):
    i = int(t*speed)
    n = 1. / (pdist(X_iter[..., i], "sqeuclidean") + 1)
    Q = n / (2.0 * np.sum(n))
    Q = squareform(Q)
    im.set_data(Q)
    return mplfig_to_npimage(f)

animation = mpy.VideoClip(make_frame_mpl, duration=X_iter.shape[2]/speed)
animation.write_videofile("similarity.webm", fps=20)

In [ ]:
mpy.ipython_display("similarity.webm", fps=20, autoplay=True)

# Dodatek A - Rozkład odległości w hiperkuli

In [ ]:
npoints = 1000
plt.figure(figsize=(15, 4))
for i, D in enumerate((2, 5, 10)):
    u = np.random.randn(npoints, D)
    u /= norm(u, axis=1)[:, None]
    r = np.random.rand(npoints, 1)
    points = u * r**(1./D)
    ax = plt.subplot(1, 3, i+1)
    ax.set_xlabel('Ball radius')
    if i == 0:
        ax.set_ylabel('Distance from origin')
    ax.hist(norm(points, axis=1),
            bins=np.linspace(0., 1., 50))
    ax.set_title('D=%d' % D, loc='left')
plt.show()

# Dodatek B - Funkcje gęstości pradwopodobieństwa

In [ ]:
z = np.linspace(0., 5., 1000)
gauss = np.exp(-z**2)
student = 1./(np.pi*(z**2 + 1))
plt.plot(z, gauss, label='Gaussian distribution')
plt.plot(z, student, label='Student-t distribution')
plt.legend()
plt.show()